# 날씨와 배추 가격은 어떤 상관 관계가 있는지 예측하는 AI를 만들어봅니다.
평균기온(avgTemp), 최저기온(minTemp), 최고기온(maxTemp), 강수량(rainFall)이 배추 가격(avePrice)에 영향을 미칠 경우 가격을 예측합니다.

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd

In [3]:
# price_data.csv 파일의 데이터를 pandas를 이용해 읽어옵니다.
price_data = pd.read_csv('price_data.csv')
print(price_data.head())   # 상위 5개만 가져온다.

       year  avgTemp  minTemp  maxTemp  rainFall  avgPrice
0  20100101     -4.9    -11.0      0.9       0.0      2123
1  20100102     -3.1     -5.5      5.5       0.8      2123
2  20100103     -2.9     -6.9      1.4       0.0      2123
3  20100104     -1.8     -5.1      2.2       5.9      2020
4  20100105     -5.2     -8.7     -1.8       0.7      2060


In [4]:
# 읽어온 데이터를 파이썬에서 처리하기 적합하게 numpy 배열 형태로 변환합니다.
data = np.array(price_data, dtype=np.float32)
print(data)
print(type(data))   # <class 'numpy.ndarray'>

[[ 2.0100100e+07 -4.9000001e+00 -1.1000000e+01  8.9999998e-01
   0.0000000e+00  2.1230000e+03]
 [ 2.0100102e+07 -3.0999999e+00 -5.5000000e+00  5.5000000e+00
   8.0000001e-01  2.1230000e+03]
 [ 2.0100104e+07 -2.9000001e+00 -6.9000001e+00  1.4000000e+00
   0.0000000e+00  2.1230000e+03]
 ...
 [ 2.0171228e+07  2.9000001e+00 -2.0999999e+00  8.0000000e+00
   0.0000000e+00  2.9010000e+03]
 [ 2.0171230e+07  2.9000001e+00 -1.6000000e+00  7.0999999e+00
   6.0000002e-01  2.9010000e+03]
 [ 2.0171232e+07  2.0999999e+00 -2.0000000e+00  5.8000002e+00
   4.0000001e-01  2.9010000e+03]]
<class 'numpy.ndarray'>


In [5]:
# numpy 배열에서 데이터 변화 요인(평균 기온, 최저 기온, 최고 기온, 강수량)으로 사용할 데이터를 뽑아냅니다.
xData = data[:, 1:5]  # 갯수는 상관없고 1행 5열   :, 뜻은 첨부터 끝까지
print(xData)

[[ -4.9 -11.    0.9   0. ]
 [ -3.1  -5.5   5.5   0.8]
 [ -2.9  -6.9   1.4   0. ]
 ...
 [  2.9  -2.1   8.    0. ]
 [  2.9  -1.6   7.1   0.6]
 [  2.1  -2.    5.8   0.4]]


In [6]:
# numpy 배열에서 결과(평균ㄱ격)로 사용할 데이터를 뽑아냅니다.
yData = data[:, 5] # 1차원 데이터로 데이터를 뽑아냅니다.
yData = data[:, [5]] # 2차원 데이터로 데이터를 뽑아냅니다. 변화 요인이 2차원이므로 결과도 2차원 형태로 뽑아내야 합니다.
print(yData)   # 1차, 2차 차이는  [12313][123123] = 1차   2차는 밑으로 쭉쭉 붙어있는거

[[2123.]
 [2123.]
 [2123.]
 ...
 [2901.]
 [2901.]
 [2901.]]


In [7]:
# 뽑아낸 데이터를 tensorflow로 처리하기 위해 placeholder를 만듭니다.
X = tf.placeholder(dtype = tf.float32, shape = [None, 4])   # 변화요인(평균기온,최저기온,최대기온,강수량) 4개     # 행은 상관없고 열4개
Y = tf.placeholder(dtype = tf.float32, shape = [None, 1])  # 가격(실제값)

# 다변인 선형 회기 모델의 기울기와 y절편을 임의의 값으로 초기화 합니다.
a = tf.Variable(tf.random_uniform([4, 1], dtype = tf.float32))  # 4행 1열을 난수로 발생시킵니다.
b = tf.Variable(tf.random_uniform([1], dtype = tf.float32))  # 값이 1개

In [8]:
# 행렬의 곱셈 연산을 이용해 다변인 선형 회귀 모델의 가설 식을 세웁니다. -> 예측값
y = tf.matmul(X, a) + b   # 행렬 * 기울기 + y절편

In [9]:
# 오차 함수를 만듭니다.
loss = tf.reduce_mean(tf.square(y - Y))

# 경사하강법 알고리즘을 사용해서 오차 함수 결과를 최소로하는 식을 만듭니다.
gradient_descent = tf.train.GradientDescentOptimizer(0.00005).minimize(loss)      
# 데이터가 많이 때문에 좀 더 세밀하게 잡아줘야해서 학습률(기울기를 0.0005만큼 쪼개서 나눈다.)을 엄청 낮게 내린다.

In [10]:
# 학습을 시킨 후 학습된 모델을 저장합니다.
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100001):
        loss_, y_, _ = sess.run([loss, y, gradient_descent], feed_dict = {X : xData, Y : yData})   # loss 값, y값, 하강법
        if i % 5000 == 0:
            print('Epoch : %6d, loss = %12.3f, price = %6.1f' % (i, loss_, y_[0]))
            
# 학습이 완료되면 학습된 모델을 디스크에 저장합니다.
# tf.train.Saver() : 학습된 모델을 디스크로 저장 또는 불러오는 객체를 생성하는 함수입니다.
    saver = tf.train.Saver()
    save_path = saver.save(sess, './saved.ckpt')  # 확장명은 ckpt
    print('학습된 모델을 저장했습니다.')

Epoch :      0, loss = 12539589.000, price =   -5.8
Epoch :   5000, loss =  2254412.000, price = 2581.4
Epoch :  10000, loss =  2228268.750, price = 2584.3
Epoch :  15000, loss =  2203933.250, price = 2587.4
Epoch :  20000, loss =  2181262.750, price = 2590.5
Epoch :  25000, loss =  2160125.250, price = 2593.8
Epoch :  30000, loss =  2140403.000, price = 2597.2
Epoch :  35000, loss =  2121989.000, price = 2600.6
Epoch :  40000, loss =  2104780.500, price = 2604.2
Epoch :  45000, loss =  2088689.500, price = 2607.8
Epoch :  50000, loss =  2073632.750, price = 2611.5
Epoch :  55000, loss =  2059533.625, price = 2615.2
Epoch :  60000, loss =  2046322.375, price = 2619.0
Epoch :  65000, loss =  2033935.250, price = 2622.8
Epoch :  70000, loss =  2022315.250, price = 2626.6
Epoch :  75000, loss =  2011406.250, price = 2630.5
Epoch :  80000, loss =  2001158.500, price = 2634.3
Epoch :  85000, loss =  1991531.000, price = 2638.2
Epoch :  90000, loss =  1982474.500, price = 2642.1
Epoch :  950